In [1]:
from dotenv import load_dotenv
from huggingface_hub import login
import os
load_dotenv()

True

In [2]:
import torch, json
from tqdm import tqdm
from PIL import Image
from unsloth import FastVisionModel
from unsloth.chat_templates import get_chat_template
from peft import PeftModel
from transformers import AutoProcessor, Florence2ForConditionalGeneration
from cragmm_search.search import UnifiedSearchPipeline
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
search_pipeline = UnifiedSearchPipeline(
    image_model_name="openai/clip-vit-large-patch14-336",
    image_hf_dataset_id="crag-mm-2025/image-search-index-validation",
    text_model_name="BAAI/bge-large-en-v1.5",
    web_hf_dataset_id="crag-mm-2025/web-search-index-validation",
)

Using device: cuda
Loading web search data from Hugging Face...


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Successfully loaded collection with 904899 entries


[urllib3.connectionpool|WARNING]Retrying (Retry(total=1, connect=1, read=2, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x726a72c9f5e0>: Failed to establish a new connection: [Errno 111] Connection refused')': /batch/
[urllib3.connectionpool|WARNING]Retrying (Retry(total=0, connect=0, read=2, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x726a72c9f790>: Failed to establish a new connection: [Errno 111] Connection refused')': /batch/
[backoff|INFO]Backing off send_request(...) for 0.1s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x726a72c9fa90>: Failed to establish a new connection: [Errno 111] Connection refused')))
[urllib3.connectionpool|WARNING]Retrying (Retry(total

Loading image index from huggingface crag-mm-2025/image-search-index-validation


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

[urllib3.connectionpool|WARNING]Retrying (Retry(total=1, connect=1, read=2, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7266e9857460>: Failed to establish a new connection: [Errno 111] Connection refused')': /batch/
[urllib3.connectionpool|WARNING]Retrying (Retry(total=0, connect=0, read=2, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7266e98578b0>: Failed to establish a new connection: [Errno 111] Connection refused')': /batch/
[backoff|INFO]Backing off send_request(...) for 0.8s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7266e9857820>: Failed to establish a new connection: [Errno 111] Connection refused')))
[urllib3.connectionpool|WARNING]Retrying (Retry(total

# Configs
Steps are defined to load/unload models in different steps. Kill the notebook process and restart the kernel process to release all memory

In [4]:
steps = ["entity_query", "crop", "task1", "web_query", "task2"]

In [5]:
raw_dataset = load_dataset("crag-mm-2025/crag-mm-single-turn-public")
# Use lazy loading to avoid running out of memory
dataset = raw_dataset['validation'].filter(lambda example: example['image'] is not None)

print(f"Original size: {len(raw_dataset['validation'])}")
print(f"Filtered size: {len(dataset)}")

Original size: 1938
Filtered size: 1423


In [6]:
total_items = len(dataset)
output_dir = "final_outputs"
use_cropped_images = True
task1_out_dir = 'task1_answers_finetune'
crop_dir = 'crop'
entity_queries_file = "entity_queries.txt"

# create output directories
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, crop_dir), exist_ok=True)
os.makedirs(os.path.join(output_dir, task1_out_dir), exist_ok=True)

starts_from_index = 0

# Fast track to skip completed steps
if os.path.exists(os.path.join(output_dir, entity_queries_file)):
    starts_from_index += 1
if os.path.exists(os.path.join(output_dir, crop_dir, f"{total_items - 1}.png")):
    starts_from_index += 1
if os.path.exists(os.path.join(output_dir, task1_out_dir, "task1_done")):
    starts_from_index += 1

print("Starting from:", steps[starts_from_index])

Starting from: task1


In [7]:
starts_from_index

2

# Generate image query

In [8]:
from unsloth.chat_templates import CHAT_TEMPLATES
print(list(CHAT_TEMPLATES.keys()))

['unsloth', 'zephyr', 'chatml', 'mistral', 'llama', 'vicuna', 'vicuna_old', 'vicuna old', 'alpaca', 'gemma', 'gemma_chatml', 'gemma2', 'gemma2_chatml', 'llama-3', 'llama3', 'phi-3', 'phi-35', 'phi-3.5', 'llama-3.1', 'llama-31', 'llama-3.2', 'llama-3.3', 'llama-32', 'llama-33', 'qwen-2.5', 'qwen-25', 'qwen25', 'qwen2.5', 'phi-4', 'gemma-3', 'gemma3', 'qwen-3', 'qwen3', 'gemma-3n', 'gemma3n', 'gpt-oss', 'gptoss', 'qwen3-instruct', 'qwen3-thinking', 'lfm-2', 'starling', 'yi-chat']


In [9]:
# Use llama 3.2 for all steps except cropping
if steps[starts_from_index] != "crop":
    # Load model with unsloth
    model, tokenizer = FastVisionModel.from_pretrained(
        model_name = 'unsloth/Llama-3.2-11B-Vision-Instruct-unsloth-bnb-4bit',
        load_in_4bit = True,
    )

    model = PeftModel.from_pretrained(model, '/home/llm/projects/CRAG/llama3.2-11B-q4-bf16')
    model = model.merge_and_unload()  # Optional: merge adapter into base

    FastVisionModel.for_inference(model)
    llama_tokenizer = get_chat_template(
        tokenizer,
        chat_template = "llama-3.2",
    )

[urllib3.connectionpool|WARNING]Retrying (Retry(total=1, connect=1, read=2, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x726a72cc02e0>: Failed to establish a new connection: [Errno 111] Connection refused')': /batch/
[urllib3.connectionpool|WARNING]Retrying (Retry(total=0, connect=0, read=2, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7266e9856890>: Failed to establish a new connection: [Errno 111] Connection refused')': /batch/
[backoff|ERROR]Giving up send_request(...) after 4 tries (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x72684e1f71f0>: Failed to establish a new connection: [Errno 111] Connection refused')))


==((====))==  Unsloth 2025.11.6: Fast Mllama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.569 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/llm/projects/CRAG/.venv/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [10]:
def generate_entity_query(image: Image, user_prompt: str) -> str:
    SYSTEM_PROMPT = (
        "You are an expert visual assistant. "
        "Given the image and the user query, identify the main subject "
        "in the image that the query refers to. "
        "Output ONLY the simple object name (e.g., 'black car', 'potted plant'). "
        "The subject always appears near the center of the image. "
        "DO NOT answer the query. Do not output a sentence. No punctuation. "
        "I REPEAT: Output ONLY the object's name, and NOTHING ELSE. "
        "Answer in less than 5 words."
    )

    content_str = f"<|image|>{user_prompt}"

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": content_str}
    ]

    prompt = tokenizer.apply_chat_template(
        messages, 
        add_generation_prompt=True,
        tokenize=False 
    )

    # In Unsloth, the tokenizer acts as the processor
    inputs = tokenizer(
        images=image,
        text=prompt,
        return_tensors="pt"
    ).to(model.device)

    output = model.generate(**inputs, max_new_tokens=5) # Short answer only
    generated_ids = output[0][inputs.input_ids.shape[1]:]
    decoded_resp = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return decoded_resp.split('\n')[0]

In [11]:
if steps[starts_from_index] == "entity_query":
    results = []

    for sample in tqdm(dataset):
        image = sample['image']
        question = sample['turns']['query'][0]

        try:
            prediction = generate_entity_query(image=image, user_prompt=question)
            results.append(prediction)
        except Exception as e:
            print(f"Error processing sample: {e}")

        if results:
            with open(os.path.join(output_dir, entity_queries_file), 'w') as f:
                f.write('\n'.join(results))
    
    starts_from_index += 1

In [12]:
starts_from_index

2

# Cropping
Define cropping function using Florence 2

In [13]:
class FlorencePipeline:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Running on device: {self.device}")

        # 1. Initialize Florence-2 (The "All-in-One" Vision Model)
        # We use 'base' for speed, or 'large' for slightly better accuracy.
        # This REPLACES both Llama 3.2 AND Grounding DINO for the cropping task.
        self.model_id = "florence-community/Florence-2-base"
        # print(f"Loading {self.model_id}...")

        # trust_remote_code=True is required for Florence-2
        self.model = Florence2ForConditionalGeneration.from_pretrained(
            self.model_id,
            trust_remote_code=True,
            dtype=torch.float16 if self.device == "cuda" else torch.float32
        ).to(self.device)

        self.processor = AutoProcessor.from_pretrained(self.model_id, use_fast=True, trust_remote_code=True)

    def ground_and_crop(self, image, query):
        """
        Uses Florence-2 to find the object described by 'query' and crop the image.
        Accepts PIL Image directly.
        """
        # print(f"-> Grounding '{query}'...")
        w, h = image.size

        # Florence-2 Task: <CAPTION_TO_PHRASE_GROUNDING>
        # We append the query to the task token
        task_prompt = "<CAPTION_TO_PHRASE_GROUNDING>"
        text_input = task_prompt + query

        # Prepare inputs (Direct PIL Image support)
        inputs = self.processor(
            text=text_input,
            images=image,
            return_tensors="pt"
        ).to(self.device, torch.float16 if self.device == "cuda" else torch.float32)

        # Generate predictions
        generated_ids = self.model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=1024,
            do_sample=False,
            num_beams=3,
        )

        # Decode output text
        generated_text = self.processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

        # Post-process to get boxes (Florence returns specific format)
        parsed_answer = self.processor.post_process_generation(
            generated_text,
            task=task_prompt,
            image_size=(image.width, image.height)
        )

        # parsed_answer is usually: {'<CAPTION_TO_PHRASE_GROUNDING>': {'bboxes': [[x1, y1, x2, y2]], 'labels': ['car']}}
        grounding_result = parsed_answer.get(task_prompt, {})
        bboxes = grounding_result.get('bboxes', [])

        if not bboxes:
            # print("-> No object found. Using full image.")
            return False, (0, 0, w, h), image

        # Take the first box (highest confidence/relevance usually first)
        box = bboxes[0]

        # Crop (Florence returns [x1, y1, x2, y2] absolute coordinates)
        # Ensure coordinates are within bounds
        x1 = max(0, box[0])
        y1 = max(0, box[1])
        x2 = min(image.width, box[2])
        y2 = min(image.height, box[3])

        if x2 <= x1 or y2 <= y1:
            # print("-> Invalid box dimensions.")
            return False, (0, 0, w, h), image

        tolerance = 10 # pixels
        is_whole_image = (x1 < tolerance and y1 < tolerance and x2 > (w - tolerance) and y2 > (h - tolerance))

        if is_whole_image:
            # print(f"-> Found object, but it spans the whole image {x1, y1, x2, y2}.")
            # We return False for 'crop success', but return the coordinates found
            return False, (x1, y1, x2, y2), image

        cropped_img = image.crop((x1, y1, x2, y2))
        return True, (x1, y1, x2, y2), cropped_img

    def run(self, image, user_query):
        # print(f"--- Processing: {user_query} ---")

        # Step 1: Ground & Crop (Florence-2)
        # Note: We pass the RAW user query directly. Florence is smart enough to extract
        # entities from questions like "Where is the black car?" -> it looks for "black car".
        cropped_image = self.ground_and_crop(image, user_query)

        # Step 2: Search (CRAG)
        search_results = search_pipeline.search(
            query_text=user_query,
            query_image=cropped_image,
            k=5
        )

        if search_results:
            print(search_results[0])

        return search_results

In [14]:
if steps[starts_from_index] == "crop":
    crop_pipeline = FlorencePipeline()

### Define prompts for cropping

This is only a simple prompt mapping based on domain of the query. Use this if image-specific prompts generated by llama3.2 are not available.

In [15]:
# Using the fact that most photos have the subject near the center, for domains with
# no specific category we use a generic prompt.
GENERIC_PROMPT = "objects, animals or people near the center of the screen"
florence_prompts = {
    1: "book",
    3: "structure",
    4: "item",
    5: "plant",
    6: "food",
    8: "building",
    9: "vehicle",
    'other': GENERIC_PROMPT
}

### Generate cropped samples

In [16]:
if steps[starts_from_index] == "crop":
    stats = []
    with open(os.path.join(output_dir, entity_queries_file), 'r') as f:
        entity_prompts = f.read().splitlines()

    for i, sample in enumerate(tqdm(dataset)):
        if not os.path.exists(os.path.join(output_dir, crop_dir, f"{i}.png")):
            domain = sample['turns']['domain'][0]
            
            success, bounds, cropped_image = crop_pipeline.ground_and_crop(sample['image'], entity_prompts[i])
            if not success:
                # Retry with generic prompt
                success, bounds, cropped_image = crop_pipeline.ground_and_crop(sample['image'], florence_prompts.get(domain, 'other'))

            cropped_image.save(os.path.join(output_dir, crop_dir, f"{i}.png"))
            stats.append(f"{i}: success [{success}] prompt [{domain}] bounding box [{bounds}]")
            with open(os.path.join(output_dir, crop_dir, "stats.txt"), "w") as f:
                f.write("\n".join(stats))

    starts_from_index += 1

In [17]:
starts_from_index

2

# Task 1
Evaluation using llama 3.2 11B model and cropped images

In [18]:
# V2 generate functions
def clean_dict(obj):
    if isinstance(obj, dict):
        cleaned = {}
        for k, v in obj.items():
            v = clean_dict(v)  # recursive clean
            # keep only non-empty and non-None values
            if v not in (None, {}, []):
                cleaned[k] = v
        return cleaned if cleaned else None
    elif isinstance(obj, list):
        cleaned_list = [clean_dict(v) for v in obj if v is not None]
        cleaned_list = [v for v in cleaned_list if v not in (None, {}, [])]
        return cleaned_list if cleaned_list else None
    else:
        return obj

def filter_and_clean(data, score_threshold=0.6):
    """Filter by score first, then clean the structure."""
    filtered = [d for d in data if d.get("score", 0) >= score_threshold]
    return [str(clean_dict(d)['entities']) for d in filtered]

def generate_answer_v2(image: Image, user_prompt: str) -> str:
    raw_results = search_pipeline(image, k=3)
    cleaned_results_list = filter_and_clean(raw_results)

    if cleaned_results_list:
        context_str = '\n'.join(cleaned_results_list)
    else:
        context_str = "No specific external information found."

    SYSTEM_PROMPT = (
        "You are a helpful assistant that truthfully answers user questions about the "
        "provided image using the provided reference information. "
        "Keep your response concise. If the answer is not in the image or references, "
        "respond with 'I don't know'."
    )
    text_payload = f"Reference Information:\n{context_str}\nUser Question: {user_prompt}"
    content_str = f"<|image|>{text_payload}"

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": content_str} # Passing string prevents TypeError
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )

    # In Unsloth, the tokenizer acts as the processor
    inputs = tokenizer(
        images=image,
        text=prompt,
        return_tensors="pt"
    ).to(model.device)

    output = model.generate(**inputs, max_new_tokens=70) # Max allowed by competition
    decoded_resp = tokenizer.decode(output[0], skip_special_tokens=True)

    # Safe extraction of the assistant's answer
    generated_ids = output[0][inputs.input_ids.shape[1]:]
    decoded_resp = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return decoded_resp

### Eval loop

In [19]:
starts_from_index = 2

In [20]:
task1_signal = os.path.exists(os.path.join(output_dir, 'task1_done'))
if steps[starts_from_index] == "task1" and not task1_signal:
    # Output file for results
    results = []

    batch_size = 10  # Renamed from 'range' (don't shadow built-in types)
    continue_from = -1 # Set to 50, 60, etc. to resume
    current_batch_start = continue_from + 1
    os.makedirs(output_dir, exist_ok=True)

    with open(os.path.join(output_dir, crop_dir, "stats.txt"), 'r') as f:
        crop_status = f.readlines()
        crop_status = [True if "[True]" in line else False for line in crop_status]

    print("Starting validation loop...")
    # Iterate through a subset for testing
    for i, sample in enumerate(tqdm(dataset)):
        # Continue from interruption
        if i <= continue_from:
            continue
        
        # Switch for cropped vs original image with fall back if cropping fails
        if use_cropped_images and crop_status[i]:
            image = Image.open(os.path.join(output_dir, crop_dir, f"{i}.png")).convert("RGB")
        else:
            image = sample.get('image', None)
        if image is None:
            continue

        question = sample['turns']['query'][0]
        ground_truth = sample['answers']['ans_full'][0]

        try:
            prediction = generate_answer_v2(image, question)
            results.append({
                "question_#": i,
                "question": question,
                "prediction": prediction,
                "ground_truth": ground_truth
            })
        except Exception as e:
            print(f"Error processing sample: {e}")

        # Batch saving results
        if (i + 1) % batch_size == 0 or i == total_items - 1:
            if len(results) > 0:
                filename = f"task1_results_{current_batch_start}-{i}.json"
                print(f"Saving {filename}...")

                out_file = os.path.join(output_dir, task1_out_dir, filename)
                with open(out_file, 'w') as f:
                    json.dump(results, f, indent=4)
                results = []
            current_batch_start = i + 1

            if i == total_items - 1:
                print("Completed all items.")
                with open(task1_signal, 'w') as f:
                    f.write('done')

starts_from_index += 1

Starting validation loop...


  0%|          | 0/1423 [00:00<?, ?it/s][urllib3.connectionpool|WARNING]Retrying (Retry(total=1, connect=1, read=2, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x726704460850>: Failed to establish a new connection: [Errno 111] Connection refused')': /batch/
[urllib3.connectionpool|WARNING]Retrying (Retry(total=0, connect=0, read=2, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x726704460be0>: Failed to establish a new connection: [Errno 111] Connection refused')': /batch/
[backoff|INFO]Backing off send_request(...) for 0.5s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x726704460c40>: Failed to establish a new connection: [Errno 111] Connection refused')))
[urllib3.conn

Saving task1_results_0-9.json...


  1%|▏         | 20/1423 [02:01<2:05:57,  5.39s/it]

Saving task1_results_10-19.json...


...

Saving task1_results_40-49.json...


  4%|▍         | 60/1423 [05:14<1:18:31,  3.46s/it]

Saving task1_results_50-59.json...


  5%|▍         | 69/1423 [06:00<1:57:55,  5.23s/it]


OSError: image file is truncated

In [ ]:
starts_from_index